In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from app.data_processing.utils import *

In [2]:
import pandas as pd

In [3]:
vg = pd.read_csv('../data/pesos_vegetales.csv', index_col = False)
ing = pd.read_csv('../data/ingredientes.csv',sep = '|')
recipes = pd.read_csv('../data/output.csv',sep='|')
bedca = pd.read_csv('../data/bedca.csv')

In [4]:
na_count = ing['Grams'].isna().groupby(ing['Recipe_id']).sum()
len(na_count[na_count == 0])


1910

In [20]:
recipes[recipes['Id']==23115]

,Id,Categoria,Nombre,Valoracion,Dificultad,Num_comensales,Tiempo,Tipo,Link_receta,Num_comentarios,Num_reviews,Fecha_modificacion,Ingredientes
8102,23115,Recetas de Ensaladas,Receta de ensalada de maíz con pollo,3.4,NaN,NaN,NaN,NaN,https://www.recetasgratis.net/receta-de-ensala...,0,4,NaN,"400 gr de pechuga de pollo,1 pimiento verde o ..."


In [17]:
def fill_grams(df):
    '''
    NaN weight values filler function.
        -The ingredients present in the weights dataset are substituted accordingly.
        -For the remaining ingredients: weight(ingredient) = mean(weight(ingredient))
    '''
    df['Index'] = df.index
    original_columns = list(df.columns)
    weights = df[df['Unidad']=='None']
    
    weights = pd.merge(weights, vg, how='inner', on='Ingrediente')

    weights['Grams'] = weights['Peso x Unidad']
    weights['Unidad'] = 'unidad'
    weights['Total_Grams'] = weights['Grams'].multiply(weights['Cantidad'], axis = 'index')
    
    
    weights = weights[original_columns]
    weights.set_index(weights['Index'], inplace=True)
    result = df.copy()
    result.loc[weights['Index'],:] = weights
    
    result[['Grams','Total_Grams']] = result.groupby(['Ingrediente'])[['Grams','Total_Grams']].transform(lambda x: x.fillna(x.mean()).round())
    na_count = result['Total_Grams'].isna().groupby(result['Recipe_id']).sum()
    
    #añadir nº comensales
    result = pd.merge(result,recipes['Num_comensales'], how='left', left_on='Recipe_id', right_index=True)
    result['Num_comensales'] = result['Num_comensales'].fillna(1)
    result['Total_Grams'] = result['Total_Grams'].divide(result['Num_comensales']).round(5)
    
    
    
    missing = list(na_count[na_count != 0].index)
    result = result[~result['Recipe_id'].isin(missing)]
    
    return result.drop(['Index'], axis=1)

In [18]:
result = fill_grams(ing)
result[result['Recipe_id']==23115]


,Recipe_id,Ingrediente,Cantidad,Unidad,Indice,Grams,Total_Grams,Num_comensales
19218,23115,pollo,400.0,pechuga,242,400.0,160000.0,1.0
19219,23115,pimiento,1.0,unidad,650,75.0,75.0,1.0
19220,23115,manzana,1.0,unidad,703,160.0,160.0,1.0
19221,23115,zumo de limón,2.0,cucharadas,834,10.0,20.0,1.0
19222,23115,maíz,400.0,lata,637,330.0,132000.0,1.0
19223,23115,zumo de pomelo,4.0,cucharadas,840,10.0,40.0,1.0
19224,23115,sal,0.5,cucharadita,908,7.0,4.0,1.0
19225,23115,azúcar,0.5,cucharadita,936,7.0,4.0,1.0
19226,23115,aceite,2.0,cucharadas,409,10.0,20.0,1.0
19227,23115,romero,NaN,ramitas,906,40.0,41.0,1.0


In [8]:
result[result['Recipe_id']==23115]

,Recipe_id,Ingrediente,Cantidad,Unidad,Indice,Grams,Total_Grams,Num_comensales
19218,23115,pollo,400.0,pechuga,242,400.0,160000.0,1.0
19219,23115,pimiento,1.0,unidad,650,75.0,75.0,1.0
19220,23115,manzana,1.0,unidad,703,160.0,160.0,1.0
19221,23115,zumo de limón,2.0,cucharadas,834,10.0,20.0,1.0
19222,23115,maíz,400.0,lata,637,330.0,132000.0,1.0
19223,23115,zumo de pomelo,4.0,cucharadas,840,10.0,40.0,1.0
19224,23115,sal,0.5,cucharadita,908,7.0,4.0,1.0
19225,23115,azúcar,0.5,cucharadita,936,7.0,4.0,1.0
19226,23115,aceite,2.0,cucharadas,409,10.0,20.0,1.0
19227,23115,romero,NaN,ramitas,906,40.0,40.0,1.0


In [525]:
na_count = result['Total_Grams'].isna().groupby(result['Recipe_id']).sum()
na_count[na_count != 0].index


Int64Index([ 6954,  7641,  7912,  8240,  8886, 11030, 12962, 13324, 13392,
            13393, 15054, 15095, 16807, 17353, 19627, 19733, 21938, 21939,
            21940, 22210, 24754, 25004, 25667, 27612, 27713, 28029, 28100,
            28543, 28721, 29356, 29723, 30783, 31701, 32861, 37744, 41354,
            41483, 43865, 43934, 50140, 50141, 50195, 51681, 52029, 52840,
            52951, 58255, 59164, 60871, 72104, 74058],
           dtype='int64', name='Recipe_id')

In [559]:
recetas = pd.read_csv('../data/output.csv',sep='|')